In [1]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
import pandas as pd

In [2]:
from utils import get_all_top_30
from utils import get_keywords
from utils import get_search_news
from database import WRITEMEDataBase

In [3]:
db = WRITEMEDataBase()

In [4]:
now = datetime.now()
date = '%04d%02d%02d' % (now.year, now.month, now.day)
time = now.strftime("%p").lower()

# 1. News Crawling

In [5]:
df = get_all_top_30()
df.groupby('category').head(1)

,title,text,url,category,date
0,"심상정 ""윤미향 해명 못 믿겠다, 민주당 책임있게 나서라"" 압박",심상정 정의당 대표가 21일 ‘윤미향 사태’와 관련해 “윤미향 당...,https://news.naver.com/main/ranking/read.nhn?r...,정치,20200521
30,'소주성'에도 빈부격차 더 커졌다…코로나 서민에 더 타격,1분기 소득 하위 20%와 상위 20%의 격차가 더 벌어졌다. ...,https://news.naver.com/main/ranking/read.nhn?r...,경제,20200521
60,"""연쇄살인 최신종, 어릴 때부터 무자비하고 여자 밝혀···형량 줄이는 법 빠삭""",신상 공개된 연쇄살인범 최신종. /김원 유튜브채널 캡쳐[서울경제] 전북...,https://news.naver.com/main/ranking/read.nhn?r...,사회,20200521
90,"中만든 90초 영상 뭐길래···트럼프 ""또라이·얼간이"" 분노 트윗",中 국영 CGTN '폼페이오 신뢰도 테스트'美 국무장관 겨냥한 인신공격...,https://news.naver.com/main/ranking/read.nhn?r...,세계,20200521
120,[취재K] “지구-소행성 충돌 가능성” 어떻게 나왔을까?,내일(22일) 오전 지구에 근접하는 소행성에 대한 관심이 뜨겁습니다....,https://news.naver.com/main/ranking/read.nhn?r...,IT/과학,20200521


# 2. Get keywords

In [6]:
keywords = df.groupby('category').apply(get_keywords)

In [7]:
keywords

category
IT/과학     [노트, 출시, 텔레콤, 회장, 요금]
경제       [주류, 코로나, 두산, 부회장, 배달]
사회       [확진, 할머니, 등교, 병원, 간호사]
세계        [중국, 일본, 백신, 코로나, 한국]
정치       [청원, 의원, 국민, 성폭행, 청와대]
dtype: object

# 3. Get hints

In [8]:
cat = '정치'
key = keywords[cat][0]

cat_df = df[df['category']==cat]
key_df = cat_df[cat_df['text'].str.contains(key)]
n = len(key_df)
search_df = get_search_news(key, news=n).iloc[:n]
hints = get_keywords(pd.concat([key_df, search_df], axis=0), only_noun=True, n=6)
if key in hints:
    hints.remove(key)
hints = hints[:5]
print(cat, key, hints)
print(key_df['title'].iloc[0])

정치 청원 ['국민', '성폭행', '청와대', '개월', '사실']
[속보]"25개월 딸 성폭행한 초등생 처벌" 청원은 '허위'


# 4. To DB

In [14]:
db.select_keywords()

()

In [15]:
db.select_hints()

()

In [16]:
cat = '정치'
db.insert_keywords(keywords[cat], cat, date, time)

In [17]:
cat = '정치'
key = keywords[cat][0]
keywordID = db.keywordID(key, db.categoryID(cat))
db.insert_hints(keywordID, hints)

In [18]:
result = db.select_keywords()
result = pd.DataFrame(result)
result

,keywordID,categoryID,keywordName,date,time
0,1,1,청원,2020-05-19,pm
1,2,1,의원,2020-05-19,pm
2,3,1,국민,2020-05-19,pm
3,4,1,성폭행,2020-05-19,pm
4,5,1,청와대,2020-05-19,pm


In [19]:
result = db.select_hints()
result = pd.DataFrame(result)
result

,keywordID,hintID,hintName
0,1,1,수업
1,1,2,학생
2,1,3,학교
3,1,4,코로나
4,1,5,교육부


In [20]:
# db.delete_keywords()
# db.delete_hints()

In [ ]:
# db.close()